# Functional Test 3.2.4 - Shared NICs with FABNetv4

This Jupyter notebook will allow you to create VMs on different sites and worker nodes consistent with requirements for test 3.2.4 for testing Shared NIC with FABNetv4 service. Because FABNetv4 works like a bridge, testing with just Shared NICs across all workers is sufficient.


## Step 1:  Configure the Environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../fablib_api/configure_environment/configure_environment.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

**This only needs to be done once.**

## Step 2: Import the FABlib Library


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

## Step 3: Check your existing slices

Since testing can get confusing, check what slices you actually have. It may print nothing if you have no active slices.

In [ ]:
try:
    for slice in fablib.get_slices():
        print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

## Step 4: Create the Test Slice

The following creates тшо nodes with Shared NICs in two selected sites on separate workers. Worker selection should be done manually. If you are unsure, the generated ads for each site ([in JSON format](https://github.com/fabric-testbed/aggregate-ads/tree/main/JSON)) can help.

**Do not change the NIC type, but you can change workers and sites**

**The code to create the slice will auto-refresh until the slice is created or it fails**

In [ ]:
from datetime import datetime
from dateutil import tz

name1='Node1'
nic1_name='SharedNIC1'

name2='Node2'
nic2_name='SharedNIC2'

# set NIC type for all NICs
nic_type='NIC_Basic'


site1='TACC' # site being tested
site2='SALT' # known good site

# worker1 goes with site1, workers 2 with site 2
worker1=f'{site1.lower()}-w1.fabric-testbed.net'
worker2=f'{site2.lower()}-w2.fabric-testbed.net'

network1_name='l3-FABNetv4-1'
network2_name='l3-FABNetv4-2'

cores=5
ram=10
disk=50
slice_name=f"Slice Test 3.2.4-SharedNIC across {site1}/{site2} on {datetime.now()}"

In [ ]:
try:
    #Create Slice
    print(f'Creating slice {slice_name}')
    slice = fablib.new_slice(name=slice_name)
    
    # Node1
    node1 = slice.add_node(name=name1, site=site1, host=worker1)
    iface1 = node1.add_component(model=nic_type, name=nic1_name).get_interfaces()[0]

    # Node2
    node2 = slice.add_node(name=name2, site=site2, host=worker2)
    iface2 = node2.add_component(model=nic_type, name=nic2_name).get_interfaces()[0]

    # Networks (one in each site)
    net1 = slice.add_l3network(name=network1_name, interfaces=[iface1], type='IPv4')
    net2 = slice.add_l3network(name=network2_name, interfaces=[iface2], type='IPv4')
 
    # Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")

## Step 5: Observe the Slice's Attributes

### Print the slice 

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

## Print the Node List

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)

    print(f"{slice.list_nodes()}")
except Exception as e:
    print(f"Exception: {e}")

## Print the Node Details

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

## Print the Interfaces

###### You should see 6 interfaces (3 cards, 2 ports).

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    
    print(f"{slice.list_interfaces()}")
except Exception as e:
    print(f"Exception: {e}")

### Shared NIC PCI Devices

Run the command <code>lspci</code> to see your Shared NIC PCI device(s).

View node1's Shared NIC - you should see one of
- `Ethernet controller: Mellanox Technologies MT28908 Family [ConnectX-6 Virtual Function]` 


In [ ]:
command = "sudo dnf install -q -y pciutils && lspci | grep Mellanox"
try:
    node1 = slice.get_node(name=name1)
    stdout, stderr = node1.execute(command)
    print(f"stdout: {stdout}")
except Exception as e:
    print(f"Exception: {e}")

## Step 6: Configure interfaces, test reachability

##  Configure IP Addresses

### Pick a Subnet

Create a subnet and list of available IP addresses. All objects are Python IP management objects. You can use either IPv4 or IPv6 subnets and addresses.

In [ ]:
try:
    network1 = slice.get_network(name=network1_name)
    network1_available_ips = network1.get_available_ips()
    print(f"{network1}")
    
    network2 = slice.get_network(name=network2_name)
    network2_available_ips =  network2.get_available_ips()
    print(f"{network2}")
except Exception as e:
    print(f"Exception: {e}")

### Configure Node1

Get the node and the interface you wish to configure.  You can use `node.get_interface` to get the interface that is connected to the specified network.  Then `pop` an IP address from the list of available IPs and call `iface.ip_addr_add` to set the IP and subnet.  

Optionally, use the `node.execute()` method to show the results of adding the IP address.

In [ ]:
try:
    node1 = slice.get_node(name=name1)        
    node1_iface = node1.get_interface(network_name=network1_name)  
    node1_addr = network1_available_ips.pop(0)
    node1_iface.ip_addr_add(addr=node1_addr, subnet=network1.get_subnet())
    
    node1.ip_route_add(subnet=network2.get_subnet(), gateway=network1.get_gateway())
    
    stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')
    print (stdout)
    
    stdout, stderr = node1.execute(f'ip route list')
    print (stdout)
except Exception as e:
    print(f"Exception: {e}")

### Configure Node2

Repeat the steps to add the next available IP to the second node.

In [ ]:
try:
    node2 = slice.get_node(name=name2)        
    node2_iface = node2.get_interface(network_name=network2_name) 
    node2_addr = network2_available_ips.pop(0)
    node2_iface.ip_addr_add(addr=node2_addr, subnet=network2.get_subnet())
    
    node2.ip_route_add(subnet=network1.get_subnet(), gateway=network2.get_gateway())
    
    stdout, stderr = node2.execute(f'ip addr show {node2_iface.get_os_interface()}')
    print (stdout)
    
    stdout, stderr = node2.execute(f'ip route list')
    print (stdout)
except Exception as e:
    print(f"Exception: {e}")

### Test reachability

Test ping between interfaces, observe successful output.


In [ ]:
try:
    node1 = slice.get_node(name=name1)        

    stdout, stderr = node1.execute(f'ping -c 5 {node2_addr}')
    print (stdout)
    print (stderr)
    
except Exception as e:
    print(f"Exception: {e}")

## Step 7: Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")